# IMPORTING LIBRARIES

In [ ]:
import pandas as pd 
import numpy as np
from haversine import haversine, Unit
import warnings
warnings.filterwarnings('ignore')
## READING DATA

In [2]:
# --- Step 1: Load the Raw Data ---
# load the three essential CSV files into pandas DataFrames.
try:
    listings_df = pd.read_csv('listings.csv')
    calendar_df = pd.read_csv('calendar.csv')
    # We only need the reviews file to get the most recent review date per listing,
    # but the listings.csv file already contains this, so we can simplify.
    # reviews_df = pd.read_csv('data/reviews.csv')
    print("Data loaded successfully.")
    print(f"Listings data has {listings_df.shape[0]} rows and {listings_df.shape[1]} columns.")
    print(f"Calendar data has {calendar_df.shape[0]} rows and {calendar_df.shape[1]} columns.")
    print("Data loaded successfully!")
except FileNotFoundError:
    print("Error: File Not Found.")
    exit() # Exit the script if files are not found


Data loaded successfully.
Listings data has 2772 rows and 79 columns.
Calendar data has 1011780 rows and 7 columns.
Data loaded successfully!


## LISTINGS

In [3]:
listings_df.head()

,id,listing_url,scrape_id,last_scraped,source,name,description,neighborhood_overview,picture_url,host_id,...,review_scores_communication,review_scores_location,review_scores_value,license,instant_bookable,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
0,21797474,https://www.airbnb.com/rooms/21797474,20250319035134,2025-03-19,city scrape,7 Southover Close Westbury On Trym Bristol,"Beautiful House 4 bedrooms, set in the beautif...",NaN,https://a0.muscache.com/pictures/865838a6-ca9a...,158665944,...,NaN,NaN,NaN,NaN,t,1,0,1,0,NaN
1,45434492,https://www.airbnb.com/rooms/45434492,20250319035134,2025-03-19,city scrape,Double room in peaceful home,NaN,NaN,https://a0.muscache.com/pictures/miso/Hosting-...,39358353,...,NaN,NaN,NaN,NaN,f,1,0,1,0,NaN
2,797137345244026756,https://www.airbnb.com/rooms/797137345244026756,20250319035134,2025-03-19,city scrape,Bright flat in central Clifton,NaN,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,27568683,...,NaN,NaN,NaN,NaN,f,1,1,0,0,NaN
3,1313695117463755982,https://www.airbnb.com/rooms/1313695117463755982,20250319035134,2025-03-19,city scrape,Clifton Village 7 bed,7 bedroom two bathroom apartment in Clifton vi...,NaN,https://a0.muscache.com/pictures/hosting/Hosti...,19404394,...,NaN,NaN,NaN,NaN,f,6,6,0,0,NaN
4,50021882,https://www.airbnb.com/rooms/50021882,20250319035134,2025-03-19,previous scrape,Bright two bedroom space with bathroom,NaN,NaN,https://a0.muscache.com/pictures/1897845b-99fa...,30844161,...,5.0,4.82,4.91,NaN,f,3,0,3,0,0.24


In [4]:
listings_df.info() # Geting a summary: how many rows, column names, data types, missing values?

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2772 entries, 0 to 2771
Data columns (total 79 columns):
 #   Column                                        Non-Null Count  Dtype  
---  ------                                        --------------  -----  
 0   id                                            2772 non-null   int64  
 1   listing_url                                   2772 non-null   object 
 2   scrape_id                                     2772 non-null   int64  
 3   last_scraped                                  2772 non-null   object 
 4   source                                        2772 non-null   object 
 5   name                                          2772 non-null   object 
 6   description                                   2726 non-null   object 
 7   neighborhood_overview                         1437 non-null   object 
 8   picture_url                                   2772 non-null   object 
 9   host_id                                       2772 non-null   i

In [5]:
listings_df.describe() # For numbers: min, max, average, etc.

,id,scrape_id,host_id,host_listings_count,host_total_listings_count,neighbourhood_group_cleansed,latitude,longitude,accommodates,bathrooms,...,review_scores_checkin,review_scores_communication,review_scores_location,review_scores_value,license,calculated_host_listings_count,calculated_host_listings_count_entire_homes,calculated_host_listings_count_private_rooms,calculated_host_listings_count_shared_rooms,reviews_per_month
count,2.772000e+03,2.772000e+03,2.772000e+03,2772.000000,2772.000000,0.0,2772.000000,2772.000000,2772.000000,2133.000000,...,2406.000000,2406.000000,2406.000000,2406.000000,0.0,2772.000000,2772.000000,2772.000000,2772.0,2406.000000
mean,6.606963e+17,2.025032e+13,1.886583e+08,29.746032,61.126263,NaN,51.461166,-2.589243,3.584416,1.313408,...,4.853516,4.856683,4.756633,4.687294,NaN,14.360029,13.237374,0.831530,0.0,1.592668
std,5.304048e+17,0.000000e+00,1.940704e+08,127.529513,343.925164,NaN,0.018158,0.027664,2.446043,0.808676,...,0.288504,0.326922,0.316506,0.391854,NaN,34.259021,32.783972,1.457236,0.0,1.708268
min,7.082000e+04,2.025032e+13,1.204320e+05,1.000000,1.000000,NaN,51.402108,-2.701430,1.000000,0.000000,...,1.000000,1.000000,1.000000,1.000000,NaN,1.000000,0.000000,0.000000,0.0,0.010000
25%,4.294845e+07,2.025032e+13,3.286254e+07,1.000000,1.000000,NaN,51.451254,-2.607932,2.000000,1.000000,...,4.830000,4.840000,4.670000,4.600000,NaN,1.000000,1.000000,0.000000,0.0,0.410000
50%,8.039701e+17,2.025032e+13,9.879705e+07,2.000000,3.000000,NaN,51.461064,-2.591810,2.000000,1.000000,...,4.940000,4.950000,4.830000,4.790000,NaN,2.000000,1.000000,0.000000,0.0,1.000000
75%,1.152724e+18,2.025032e+13,3.409946e+08,11.000000,14.000000,NaN,51.470475,-2.576097,4.000000,1.500000,...,5.000000,5.000000,4.950000,4.910000,NaN,6.000000,4.000000,1.000000,0.0,2.210000
max,1.379139e+18,2.025032e+13,6.847737e+08,3262.000000,8645.000000,NaN,51.513480,-2.511810,16.000000,15.000000,...,5.000000,5.000000,5.000000,5.000000,NaN,148.000000,140.000000,9.000000,0.0,13.250000


In [6]:
listings_df.isnull().sum() # Counting how many missing values in each column

id                                                0
listing_url                                       0
scrape_id                                         0
last_scraped                                      0
source                                            0
                                               ... 
calculated_host_listings_count                    0
calculated_host_listings_count_entire_homes       0
calculated_host_listings_count_private_rooms      0
calculated_host_listings_count_shared_rooms       0
reviews_per_month                               366
Length: 79, dtype: int64

### SELECTING COLUMNS FROM LISTINGS DATA FOR FINAL MODELLING

In [7]:
# Keep only the columns that will be useful as features for the model
columns_to_keep = [
    'id', 'host_is_superhost', 'host_response_rate', 'host_identity_verified',
    'neighbourhood_cleansed', 'latitude', 'longitude', 'property_type', 'room_type',
    'accommodates', 'bathrooms_text', 'bedrooms', 'beds', 'amenities',
    'review_scores_rating', 'number_of_reviews', 'last_review'
]
listings_prepared_df = listings_df[columns_to_keep].copy()

### LISTING DATA CLEANING

In [8]:
# Rename 'id' to 'listing_id' to make merging easier
listings_prepared_df.rename(columns={'id': 'listing_id'}, inplace=True)

# Clean and convert boolean-like columns ('t'/'f') to 1s and 0s
for col in ['host_is_superhost', 'host_identity_verified']:
    listings_prepared_df[col] = np.where(listings_prepared_df[col] == 't', 1, 0)

# Clean host_response_rate (e.g., '95%') to a number (e.g., 0.95)
listings_prepared_df['host_response_rate'] = listings_prepared_df['host_response_rate'].str.replace('%', '', regex=False).astype(float) / 100

# Extract number of bathrooms from 'bathrooms_text' (e.g., "1.5 baths" -> 1.5)
listings_prepared_df['bathrooms'] = listings_prepared_df['bathrooms_text'].str.extract('(\d+\.?\d*)').astype(float)

print("\nListings data cleaned and prepared.")


Listings data cleaned and prepared.


## CALENDAR

In [9]:
calendar_df.head()

,listing_id,date,available,price,adjusted_price,minimum_nights,maximum_nights
0,70820,19-03-2025,f,$40.00,NaN,7,14
1,70820,20-03-2025,f,$40.00,NaN,7,14
2,70820,21-03-2025,f,$40.00,NaN,7,14
3,70820,22-03-2025,f,$40.00,NaN,7,14
4,70820,23-03-2025,f,$40.00,NaN,7,14


In [10]:
calendar_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011780 entries, 0 to 1011779
Data columns (total 7 columns):
 #   Column          Non-Null Count    Dtype  
---  ------          --------------    -----  
 0   listing_id      1011780 non-null  int64  
 1   date            1011780 non-null  object 
 2   available       1011780 non-null  object 
 3   price           1011780 non-null  object 
 4   adjusted_price  0 non-null        float64
 5   minimum_nights  1011780 non-null  int64  
 6   maximum_nights  1011780 non-null  int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 54.0+ MB


In [11]:
calendar_df.describe()

,listing_id,adjusted_price,minimum_nights,maximum_nights
count,1.011780e+06,0.0,1.011780e+06,1.011780e+06
mean,6.606963e+17,NaN,3.930505e+00,5.504178e+02
std,5.303094e+17,NaN,1.088609e+01,4.543097e+02
min,7.082000e+04,NaN,1.000000e+00,1.000000e+00
25%,4.294845e+07,NaN,1.000000e+00,9.000000e+01
50%,8.039701e+17,NaN,2.000000e+00,3.650000e+02
75%,1.152724e+18,NaN,3.000000e+00,1.125000e+03
max,1.379139e+18,NaN,3.650000e+02,1.125000e+03


In [12]:
calendar_df.isnull().sum()

listing_id              0
date                    0
available               0
price                   0
adjusted_price    1011780
minimum_nights          0
maximum_nights          0
dtype: int64

### CLEANING CALENDAR DATA

In [13]:
# Convert the 'date' column to a proper datetime format
calendar_df['date'] = pd.to_datetime(calendar_df['date'])

# Clean the 'price' column: remove '$', ',', and convert to a number (float)
# We use .str.replace with a regular expression to remove non-digit characters
calendar_df['price'] = calendar_df['price'].replace({'\$': '', ',': ''}, regex=True).astype(float)

# Create the target variable 'is_booked'
# The logic is: if a date is NOT available ('f'), we infer it is booked.
# 't' (true) -> 0 (not booked)
# 'f' (false) -> 1 (booked)
calendar_df['is_booked'] = np.where(calendar_df['available'] == 'f', 1, 0)

# We only need the id, date, price, and our new target variable
calendar_prepared_df = calendar_df[['listing_id', 'date', 'price', 'is_booked']].copy()


## MERGING LISTINGS WITH CALENDAR

In [14]:
# --- Merge DataFrames ---
# combining the calendar and listings data into one master table.
print("\nMerging calendar and listings data...")
master_df = pd.merge(calendar_prepared_df, listings_prepared_df, on='listing_id', how='left')
print("Merge complete.")
print(f"Master table has {master_df.shape[0]} rows and {master_df.shape[1]} columns.")



Merging calendar and listings data...
Merge complete.
Master table has 1011780 rows and 21 columns.


## VERIFYING MERGED DATA

In [15]:
master_df.head()

,listing_id,date,price,is_booked,host_is_superhost,host_response_rate,host_identity_verified,neighbourhood_cleansed,latitude,longitude,...,room_type,accommodates,bathrooms_text,bedrooms,beds,amenities,review_scores_rating,number_of_reviews,last_review,bathrooms
0,70820,2025-03-19,40.0,1,1,1.0,1,Windmill Hill,51.43843,-2.59183,...,Private room,2,1 shared bath,1.0,1.0,"[""Oven"", ""Lockbox"", ""Heating"", ""Hot water"", ""C...",4.78,197,2024-12-12,1.0
1,70820,2025-03-20,40.0,1,1,1.0,1,Windmill Hill,51.43843,-2.59183,...,Private room,2,1 shared bath,1.0,1.0,"[""Oven"", ""Lockbox"", ""Heating"", ""Hot water"", ""C...",4.78,197,2024-12-12,1.0
2,70820,2025-03-21,40.0,1,1,1.0,1,Windmill Hill,51.43843,-2.59183,...,Private room,2,1 shared bath,1.0,1.0,"[""Oven"", ""Lockbox"", ""Heating"", ""Hot water"", ""C...",4.78,197,2024-12-12,1.0
3,70820,2025-03-22,40.0,1,1,1.0,1,Windmill Hill,51.43843,-2.59183,...,Private room,2,1 shared bath,1.0,1.0,"[""Oven"", ""Lockbox"", ""Heating"", ""Hot water"", ""C...",4.78,197,2024-12-12,1.0
4,70820,2025-03-23,40.0,1,1,1.0,1,Windmill Hill,51.43843,-2.59183,...,Private room,2,1 shared bath,1.0,1.0,"[""Oven"", ""Lockbox"", ""Heating"", ""Hot water"", ""C...",4.78,197,2024-12-12,1.0


In [16]:
master_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011780 entries, 0 to 1011779
Data columns (total 21 columns):
 #   Column                  Non-Null Count    Dtype         
---  ------                  --------------    -----         
 0   listing_id              1011780 non-null  int64         
 1   date                    1011780 non-null  datetime64[ns]
 2   price                   1011780 non-null  float64       
 3   is_booked               1011780 non-null  int64         
 4   host_is_superhost       1011780 non-null  int64         
 5   host_response_rate      799350 non-null   float64       
 6   host_identity_verified  1011780 non-null  int64         
 7   neighbourhood_cleansed  1011780 non-null  object        
 8   latitude                1011780 non-null  float64       
 9   longitude               1011780 non-null  float64       
 10  property_type           1011780 non-null  object        
 11  room_type               1011780 non-null  object        
 12  accommodates  

In [17]:
master_df.isnull().sum() # Checking missing values after merge

listing_id                     0
date                           0
price                          0
is_booked                      0
host_is_superhost              0
host_response_rate        212430
host_identity_verified         0
neighbourhood_cleansed         0
latitude                       0
longitude                      0
property_type                  0
room_type                      0
accommodates                   0
bathrooms_text               730
bedrooms                   75190
beds                      233965
amenities                      0
review_scores_rating      133590
number_of_reviews              0
last_review               133590
bathrooms                  11315
dtype: int64

## FEATURE ENGINEERING

In [18]:
# --- Time Features ---
master_df['day_of_week'] = master_df['date'].dt.dayofweek # Monday=0, Sunday=6
master_df['month'] = master_df['date'].dt.month
master_df['year'] = master_df['date'].dt.year
master_df['is_weekend'] = np.where(master_df['day_of_week'].isin([5, 6]), 1, 0)

# --- Bank Holiday Feature ---
uk_bank_holidays = [
    '2025-01-01', '2025-04-18', '2025-04-21', '2025-05-05', '2025-05-26',
    '2025-08-25', '2025-12-25', '2025-12-26', '2026-01-01', '2026-04-03',
    '2026-04-06', '2026-05-04', '2026-05-25', '2026-08-31', '2026-12-25', '2026-12-28'
]
master_df['is_bank_holiday'] = master_df['date'].dt.strftime('%Y-%m-%d').isin(uk_bank_holidays).astype(int)

# --- Location Features ---
# Define Bristol's city centre coordinates (approximated near Bristol Cathedral)
bristol_centre = (51.4519, -2.5999)
# Calculate distance for each property to the city centre in MILES
master_df['distance_to_centre_miles'] = master_df.apply(
    lambda row: haversine((row['latitude'], row['longitude']), bristol_centre, unit=Unit.MILES),
    axis=1
)

# --- Property Features ---
# Count the number of amenities for each property
master_df['amenities_count'] = master_df['amenities'].apply(lambda x: len(x.split(',')))

# --- Review Features ---
# Calculate review recency (how many days ago was the last review?)
master_df['last_review'] = pd.to_datetime(master_df['last_review'])
# We use a fixed date for calculation to ensure consistency
snapshot_date = pd.to_datetime('2025-03-18') # The date the data was scraped
master_df['review_recency_days'] = (snapshot_date - master_df['last_review']).dt.days

# Now remove the last_review feature
master_df = master_df.drop('last_review', axis = 1)

print("\nFeature engineering complete. New features created.")


Feature engineering complete. New features created.


## FINAL CLEANING, ENCODING AND COLUMN SELECTION

In [19]:
# --- Final Cleaning, Encoding, and Column Selection ---

# Handle any missing values that might exist after the merge
# For simplicity, we'll fill numerical NaNs with the median value of their column
for col in ['host_response_rate', 'bedrooms', 'beds', 'bathrooms', 'review_scores_rating', 'review_recency_days']:
    master_df[col].fillna(master_df[col].median(), inplace=True)

# One-Hot Encode categorical variables like 'room_type' and 'neighbourhood_cleansed'
# This converts text categories into numerical columns (0s and 1s)
master_df = pd.get_dummies(master_df, columns=['room_type', 'neighbourhood_cleansed'], dummy_na=False)

# Select the final list of columns for our model
# We exclude raw/intermediate columns like lat/lon, dates, text, etc.
final_feature_columns = [
    'price', 'accommodates', 'bathrooms', 'bedrooms', 'beds',
    'host_is_superhost', 'host_response_rate', 'host_identity_verified',
    'review_scores_rating', 'number_of_reviews', 'day_of_week', 'month', 'year',
    'is_weekend', 'is_bank_holiday', 'distance_to_centre_miles', 'amenities_count',
    'review_recency_days', 'date', 'listing_id'
]
# Add the one-hot encoded columns to our feature list
final_feature_columns += [col for col in master_df.columns if 'room_type_' in col or 'neighbourhood_cleansed_' in col]

# The final DataFrame for modelling
final_df = master_df[final_feature_columns + ['is_booked']].copy()

## FINAL CHECK

In [20]:
final_df.head()

,price,accommodates,bathrooms,bedrooms,beds,host_is_superhost,host_response_rate,host_identity_verified,review_scores_rating,number_of_reviews,...,neighbourhood_cleansed_Southmead,neighbourhood_cleansed_Southville,neighbourhood_cleansed_St George Central,neighbourhood_cleansed_St George Troopers Hill,neighbourhood_cleansed_St George West,neighbourhood_cleansed_Stockwood,neighbourhood_cleansed_Stoke Bishop,neighbourhood_cleansed_Westbury-on-Trym & Henleaze,neighbourhood_cleansed_Windmill Hill,is_booked
0,40.0,2,1.0,1.0,1.0,1,1.0,1,4.78,197,...,False,False,False,False,False,False,False,False,True,1
1,40.0,2,1.0,1.0,1.0,1,1.0,1,4.78,197,...,False,False,False,False,False,False,False,False,True,1
2,40.0,2,1.0,1.0,1.0,1,1.0,1,4.78,197,...,False,False,False,False,False,False,False,False,True,1
3,40.0,2,1.0,1.0,1.0,1,1.0,1,4.78,197,...,False,False,False,False,False,False,False,False,True,1
4,40.0,2,1.0,1.0,1.0,1,1.0,1,4.78,197,...,False,False,False,False,False,False,False,False,True,1


In [21]:
master_df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1011780 entries, 0 to 1011779
Data columns (total 63 columns):
 #   Column                                              Non-Null Count    Dtype         
---  ------                                              --------------    -----         
 0   listing_id                                          1011780 non-null  int64         
 1   date                                                1011780 non-null  datetime64[ns]
 2   price                                               1011780 non-null  float64       
 3   is_booked                                           1011780 non-null  int64         
 4   host_is_superhost                                   1011780 non-null  int64         
 5   host_response_rate                                  1011780 non-null  float64       
 6   host_identity_verified                              1011780 non-null  int64         
 7   latitude                                            1011780 non-null  fl

## SAVE THE FINAL DATA TO CSV

In [22]:
final_df.to_csv('final_airbnb_data.csv', index = False)

# LOADING CLEANED DATASET

In [23]:
df = pd.read_csv('final_airbnb_data.csv')
df.head()

,price,accommodates,bathrooms,bedrooms,beds,host_is_superhost,host_response_rate,host_identity_verified,review_scores_rating,number_of_reviews,...,neighbourhood_cleansed_Southmead,neighbourhood_cleansed_Southville,neighbourhood_cleansed_St George Central,neighbourhood_cleansed_St George Troopers Hill,neighbourhood_cleansed_St George West,neighbourhood_cleansed_Stockwood,neighbourhood_cleansed_Stoke Bishop,neighbourhood_cleansed_Westbury-on-Trym & Henleaze,neighbourhood_cleansed_Windmill Hill,is_booked
0,40.0,2,1.0,1.0,1.0,1,1.0,1,4.78,197,...,False,False,False,False,False,False,False,False,True,1
1,40.0,2,1.0,1.0,1.0,1,1.0,1,4.78,197,...,False,False,False,False,False,False,False,False,True,1
2,40.0,2,1.0,1.0,1.0,1,1.0,1,4.78,197,...,False,False,False,False,False,False,False,False,True,1
3,40.0,2,1.0,1.0,1.0,1,1.0,1,4.78,197,...,False,False,False,False,False,False,False,False,True,1
4,40.0,2,1.0,1.0,1.0,1,1.0,1,4.78,197,...,False,False,False,False,False,False,False,False,True,1
